## **Part 3: Estimation**

In [10]:
# Plotting
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
pio.renderers.default = "notebook+pdf"
pio.templates.default = "plotly_dark"

# Utilities
import numpy as np
from tqdm import tqdm

### **Task 12**

In [11]:
num_women = 1000

# double check if this is the right Q matrix
Q = np.array([[0, 0.0025, 0.00125, 0, 0.001],
              [0, 0, 0, 0.002, 0.005],
              [0, 0, 0, 0.003, 0.005],
              [0, 0, 0, 0, 0.009],
              [0, 0, 0, 0, 0]])

for i in range(Q.shape[0]):
    Q[i,i] = -np.sum(Q[i,:])

print(Q)

[[-0.00475  0.0025   0.00125  0.       0.001  ]
 [ 0.      -0.007    0.       0.002    0.005  ]
 [ 0.       0.      -0.008    0.003    0.005  ]
 [ 0.       0.       0.      -0.009    0.009  ]
 [ 0.       0.       0.       0.      -0.     ]]


In [12]:
def changeState(Q, state, currentTime, endTime):
    if state == 4:
        return endTime, 4
    qii = Q[state,state]
    if qii == 0:
        return endTime, state
    time = np.random.exponential(-(1/qii))
    if currentTime + time > endTime:
        return endTime - currentTime, state
    # Determine the next state
    ps = np.copy(Q[state, :])
    ps[state] = 0
    ps /= -qii
    nextState = np.random.choice(range(5), p=ps)
    return time, nextState

def simulateWoman(Q, startingState, endTime):
    state = startingState
    time = 0
    times = [0]
    states = [state]
    while state != 4 and time < endTime:
        dt, state = changeState(Q, state, time, endTime)
        time += dt
        times.append(time)
        states.append(state)
    return np.array(times), np.array(states)

def simulateWomen(Q, n, startingStates, endTime):
    women = []
    for i in range(n):
        times, states = simulateWoman(Q, startingStates[i], endTime)
        women.append((times, states))
    return women

endTime = np.inf
startingStates = np.zeros(num_women, dtype=int)
women = simulateWomen(Q, num_women, startingStates, endTime)


In [13]:
max_time = 0
for i in range(num_women):
    temp = women[i][0][-1]

    if temp > max_time:
        max_time = temp

time_steps = np.ceil(max_time/48)

Y = np.zeros((num_women, int(time_steps+1)), dtype=int)

for i in range(num_women):
    for j in range(int(time_steps+1)):
        true_indices = np.where([women[i][0] <= 48*j])[1]
        if true_indices.size == 0:
            Y[i,j] = 0
        else:
            Y[i,j] = women[i][1][true_indices.max()]


### **Task 13**

In [41]:
def est_Q(Q_0, Y):
    Q = Q_0
    err = 1

    time_steps = Y.shape[1]
    num_women = Y.shape[0]

    k = 0

    while np.abs(err) > 1e-3:

        k = k + 1
        print(f"Iteration: {k}")

        N = np.zeros((5,5))
        S = np.zeros((5))

        Q_old = np.copy(Q)

        for i in tqdm(range(num_women)):
            for j in range(1, int(time_steps)):
                #print(j)
                while True:
                    women = simulateWomen(Q_old, 1, [Y[i,j-1]], 48)

                    Y_new_state = women[0][1]
                    Y_new_time = women[0][0]

                    if Y_new_state[-1] == Y[i,j]:
                        for k in range(1,len(Y_new_state)):
                            N[Y_new_state[k-1],Y_new_state[k]] += 1
                            S[Y_new_state[k]] += Y_new_time[k]
                        break

        Q = np.zeros((5,5))

        for i in range(5):
            for j in range(5):
                if i != j:
                    Q[i,j] = N[i,j]/S[i]

            Q[i,i] = -np.sum(Q[i,:])

        err = np.linalg.norm(Q_old - Q)

    return Q

def generate_matrix():
    # Generate a 5x5 matrix with random positive elements
    matrix = np.random.rand(5, 5)
    matrix /= 100

    # Make the lower triangular part zero
    matrix = np.triu(matrix)

    # Make the diagonal elements the negative sum of the remaining row elements
    np.fill_diagonal(matrix, -matrix.sum(axis=1) + np.diag(matrix))

    return matrix

Q_01 = generate_matrix()
Q_02 = np.copy(Q)

Q2 = est_Q(Q_02, Y)
Q1 = est_Q(Q_01, Y)


Iteration: 1


100%|██████████| 1000/1000 [00:00<00:00, 1681.04it/s]


Iteration: 3


100%|██████████| 1000/1000 [00:00<00:00, 1102.58it/s]


Iteration: 2


...

100%|██████████| 1000/1000 [00:00<00:00, 1518.58it/s]


Iteration: 2


100%|██████████| 1000/1000 [00:00<00:00, 1774.64it/s]


Iteration: 3


100%|██████████| 1000/1000 [00:00<00:00, 1800.25it/s]


In [42]:
print("True matrix")
print(Q)

print("From random matrix")
with np.printoptions(suppress=True):
    print(np.round(Q1, 5))

print("From true matrix")
print(np.round(Q2, 5))

print(np.linalg.norm(Q1 - Q2))

True matrix
[[-0.00475  0.0025   0.00125  0.       0.001  ]
 [ 0.      -0.007    0.       0.002    0.005  ]
 [ 0.       0.      -0.008    0.003    0.005  ]
 [ 0.       0.       0.      -0.009    0.009  ]
 [ 0.       0.       0.       0.      -0.     ]]
From random matrix
[[-0.00555  0.00296  0.00131  0.00004  0.00124]
 [ 0.      -0.00618  0.00002  0.00165  0.00451]
 [ 0.       0.      -0.00657  0.00232  0.00425]
 [ 0.       0.       0.      -0.00686  0.00686]
 [ 0.       0.       0.       0.      -0.     ]]
From true matrix
[[-0.00555  0.003    0.00135  0.       0.00119]
 [ 0.      -0.00628  0.       0.00172  0.00456]
 [ 0.       0.      -0.00671  0.00236  0.00434]
 [ 0.       0.       0.      -0.00693  0.00693]
 [ 0.       0.       0.       0.      -0.     ]]
0.00025711842341959596
